In [1]:
import pandas as pd
from Bio import SeqIO 
import os,sys
import warnings   
warnings.filterwarnings('ignore')
import configparser
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import argparse   
import json
from os.path import exists,splitext,dirname,splitext,basename,realpath,abspath
from Bio import SeqIO

In [ ]:
# test


In [ ]:
#计算A-B之间的快递费用
1.快递费用按计费重量W计算，计费重量以KG为单位，实际重量不满1KG的部分记为1KG
2.1KG(含)以内18元，包含运费和保险费;
3.每增加1KG，运费增加5元，保险费为上一档费用(运费+保险费)的1%;
4。每档快递费用计算结果按照四舍五入取整数值
5.最大计费重量为100KG


In [1]:
import math
import random
from datetime import datetime
import pandas as pd
import pandas as pd
import sqlite3



def A_B_cost(weight):
    cost = 0 
    if weight <= 1:
        cost = 18
    elif weight <=100:
        cost = 18
        remainder = weight - math.floor(weight)

        for i in range(1,  math.floor(weight) ):
            #保险费：上一档费用的1%
            insurance_cost = cost*0.01
            
            freight_cost = 5
            cost = round(cost + freight_cost + insurance_cost)

        if remainder != 0:
            cost = cost + 18
    else:
        print("超过100KG")
        return None
    return cost

def generate_ids(num_ids):
    user_ids = random.sample(range(1, 1000000), num_ids)
    return user_ids

def generate_orders(order_ids, user_ids):
    num_orders = len(order_ids)

    orders = []
    weights = [1/w for w in range(1, 101)]  # 生成权重列表，范围为1到100KG
    total_weight = sum(weights)
    probabilities = [weight/total_weight for weight in weights]  # 计算每个重量的概率

    for id in order_ids:

        weight = random.choices(range(1, 101), probabilities)[0]  # 按概率选择重量
        user_id = random.choices(user_ids)[0]
        now = datetime.now()
        order = {
            "id":id,
            "uid":user_id,
            "weight":weight,
            "create_at":now
        }

        orders.append(order)
        
    return orders

def insert_user_order(df, db_name='orders.db', table_name='user_order'):
    # 连接到SQLite数据库
    conn = sqlite3.connect(db_name)  
    try:
         # 将DataFrame存入数据库，如果表不存在则创建新表
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print("写入数据库成功！")
    except Exception as e:
         print("写入数据库时出现错误:", e)

    # 关闭数据库连接
    conn.close()

def get_user_orders(user_id, db_name='orders.db', table_name="user_order"):
    # 连接到数据库
    db_connection = sqlite3.connect(db_name)

    # 查询用户的所有订单
    sql = f"SELECT * FROM {table_name} WHERE uid = ?"
    print(sql)  
    orders_cursor  = db_connection.execute(sql, (user_id,))

    #显示用户的订单
    user_orders = orders_cursor.fetchall()
    

    # 关闭数据库连接
    db_connection.close()

    return user_orders  

def main(user_id):
    #1.使用sqlite数据库存储订单数据
    user_ids = generate_ids(1000)   # 生成 1000 个不重复的用户 ID
    order_ids = generate_ids(100000)   # 生成 100000 个不重复的订单 ID
    user_order_tb = pd.DataFrame( generate_orders(order_ids, user_ids)  )

    insert_user_order(user_order_tb, db_name='orders.db', table_name='user_order')

    #2.根据uid查询用户订单
    user_orders = get_user_orders(user_id)
   
    if user_orders != []:
        column = ["id",	"uid",	"weight",	"create_at"]
        user_orders_df = pd.DataFrame(user_orders,columns=column)

        #3.计算某用户订单的和
        total_cost = user_orders_df["weight"].sum()
        print(f"用户{user_id}所有订单的总费用：{total_cost}")
    else:
        print(f"用户id：{user_id}，该用户不存在")



In [2]:
main(677546)

写入数据库成功！
SELECT * FROM user_order WHERE uid = ?
用户id：677546，该用户不存在


In [12]:
db_connection = sqlite3.connect("orders.db")

In [13]:
cursor =   db_connection.cursor()

In [14]:
cursor.fetchall()

[]

In [44]:
user_order_tb

,id,uid,weight,create_at
0,353437,36098,4,2024-03-23 20:54:26.062721
1,900402,236550,39,2024-03-23 20:54:26.062742
2,192267,774661,5,2024-03-23 20:54:26.062751
3,508060,325755,37,2024-03-23 20:54:26.062759
4,577131,83963,1,2024-03-23 20:54:26.062766
...,...,...,...,...
99995,918211,962287,2,2024-03-23 20:54:26.736927
99996,509836,243544,3,2024-03-23 20:54:26.736934
99997,272503,662181,8,2024-03-23 20:54:26.736945
99998,960185,89023,1,2024-03-23 20:54:26.736952


In [6]:
#1.使用sqlite数据库存储订单数据
user_ids = generate_ids(1000)   # 生成 1000 个不重复的用户 ID
order_ids = generate_ids(100000)   # 生成 100000 个不重复的订单 ID
user_order_tb = pd.DataFrame( generate_orders(order_ids, user_ids)  )

# 连接到SQLite数据库
db_name = "orders.db"
conn = sqlite3.connect(db_name)  
try:
    # 将DataFrame存入数据库，如果表不存在则创建新表
    a=user_order_tb.to_sql("user_order", conn, index=False, if_exists='replace')
    print("写入数据库成功！")
except Exception as e:
    print("写入数据库时出现错误:", e)

# 创建游标对象
cursor = conn.cursor()

sql = f"SELECT * FROM user_order WHERE weight = ?"
print(sql)  
orders_cursor = cursor.execute(sql, (168308,))
orders_cursor.fetchall()

写入数据库成功！
SELECT * FROM user_order WHERE weight = ?


[]

In [7]:
user_order_tb

,id,uid,weight,create_at
0,592312,518376,4,2024-03-23 21:03:55.319588
1,274198,817483,27,2024-03-23 21:03:55.319606
2,13202,742076,6,2024-03-23 21:03:55.319615
3,840001,19761,46,2024-03-23 21:03:55.319623
4,348246,728820,1,2024-03-23 21:03:55.319631
...,...,...,...,...
99995,193624,920647,1,2024-03-23 21:03:55.972807
99996,594768,883469,8,2024-03-23 21:03:55.972813
99997,686728,447398,1,2024-03-23 21:03:55.972819
99998,514485,325637,2,2024-03-23 21:03:55.972825


In [10]:
# 连接到数据库
conn = sqlite3.connect("orders.db")

# 创建游标对象
cursor = conn.cursor()

cursor.execute("SELECT * FROM user_order where uid=883469")

result = cursor.fetchall()

In [11]:
result

[(102279, 883469, 12, '2024-03-23 21:03:55.330107'),
 (25108, 883469, 93, '2024-03-23 21:03:55.331044'),
 (364697, 883469, 12, '2024-03-23 21:03:55.336363'),
 (31072, 883469, 8, '2024-03-23 21:03:55.338222'),
 (607059, 883469, 12, '2024-03-23 21:03:55.338422'),
 (301973, 883469, 50, '2024-03-23 21:03:55.346238'),
 (149721, 883469, 2, '2024-03-23 21:03:55.360375'),
 (588333, 883469, 55, '2024-03-23 21:03:55.361579'),
 (531504, 883469, 63, '2024-03-23 21:03:55.363365'),
 (442887, 883469, 1, '2024-03-23 21:03:55.370393'),
 (185682, 883469, 24, '2024-03-23 21:03:55.375671'),
 (375566, 883469, 58, '2024-03-23 21:03:55.378131'),
 (550163, 883469, 4, '2024-03-23 21:03:55.380687'),
 (601769, 883469, 7, '2024-03-23 21:03:55.390470'),
 (970931, 883469, 7, '2024-03-23 21:03:55.390557'),
 (931042, 883469, 3, '2024-03-23 21:03:55.394536'),
 (695358, 883469, 3, '2024-03-23 21:03:55.394567'),
 (106587, 883469, 4, '2024-03-23 21:03:55.396425'),
 (643803, 883469, 24, '2024-03-23 21:03:55.401474'),
 (31

In [ ]:
db_connection.close()

In [27]:
# 获取用户订单
user_orders = get_user_orders(11)

AttributeError: 'sqlite3.Connection' object has no attribute 'fetchall'

In [ ]:
import sqlite3
from datetime import datetime

conn = sqlite3.connect('orders.db')

# 创建一个游标对象
cursor = conn.cursor()

# 创建订单数据表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        id INTEGER PRIMARY KEY,
        uid INTEGER,
        weight REAL,
        created_at DATETIME
    )
''')

# 示例订单数据
order_data = {
    "id":2,
    'uid': 1,
    'weight': 5.7,
    'created_at': datetime.now()
}




# 插入订单数据
cursor.execute('''
    INSERT INTO orders (id, uid, weight, created_at)
    VALUES (?, ?, ?, ?)
''', (order_data["id"] ,order_data['uid'], order_data['weight'], order_data['created_at']))

# 提交更改
conn.commit()

# 关闭连接
conn.close()


In [6]:
A_B_cost(50)

341

In [3]:

# 计算总订单数量
total_orders = 100000

weights = [1/w for w in range(1, 101)]  # 生成权重列表，范围为1到100KG


total_weight = sum(weights)
probabilities = [weight/total_weight for weight in weights]  # 计算每个重量分布的概率







In [1]:
len("ACTCGAGAGGGCACTCTAGTAGATAGGTCTTTACTGATTTGGAGCGGATCTGGAAGAATATGAGCTATGGTGTGTTTTCAACTTGTAGATGACGTTTTGCTTGGATCTGGTGCCGTGCAGTCGGGCAAATTGAAGGTGTTCAGTTCTTGGAGTTGCCTTTTTaggagtacagtatctacagtagtgtCTGTCTAGTGTCTGGTGTCTACTGTAGTGAGTGCACTTTCAAGCCGTCTGTTTCAATTGAATGACATAGACCAGAGTCATGAGCATCAGAGATCGCTATATGGATATCTTTATTGGGTCGCTTCACATGTCTACGGTACTCTCCGCAGATGATTCAGTTACTCCCATTCCTATCCCCCATTCCTATCCCCCATCATTCGATTCACCCACCATGCCACCTCCGAGGTTATCCACTCTATACAGGGTTTGTTGAGACAGTCTGGGTCCGAGGTGGCTACACCTTCTTTGTACTGAGCCCAAGTGGAATTTTAGCC")

500

In [ ]:
random.choices(range(1, 101), probabilities)




In [1]:
import random

def generate_orders(num_orders):
    orders = []
    weights = [1/w for w in range(1, 101)]  # 生成权重列表，范围为1到100KG
    total_weight = sum(weights)
    probabilities = [weight/total_weight for weight in weights]  # 计算每个重量的概率

    for _ in range(num_orders):
        weight = random.choices(range(1, 101), probabilities)[0]  # 按概率选择重量
        orders.append(weight)
        
    return orders

# 生成 100000 条订单
orders = generate_orders(100000)




In [16]:


# 统计订单重量的分布情况
weight_counts = {}
for weight in orders:
    weight_counts[weight] = weight_counts.get(weight, 0) + 1

# 输出订单重量的分布情况
for weight, count in sorted(weight_counts.items()):
    print(f"订单重量为 {weight}KG 的数量为 {count}")

# 输出订单总数
print("总订单数:", len(orders))

订单重量为 1KG 的数量为 19146
订单重量为 2KG 的数量为 9818
订单重量为 3KG 的数量为 6547
订单重量为 4KG 的数量为 4797
订单重量为 5KG 的数量为 3785
订单重量为 6KG 的数量为 3248
订单重量为 7KG 的数量为 2773
订单重量为 8KG 的数量为 2414
订单重量为 9KG 的数量为 2144
订单重量为 10KG 的数量为 1966
订单重量为 11KG 的数量为 1791
订单重量为 12KG 的数量为 1627
订单重量为 13KG 的数量为 1503
订单重量为 14KG 的数量为 1377
订单重量为 15KG 的数量为 1315
订单重量为 16KG 的数量为 1178
订单重量为 17KG 的数量为 1169
订单重量为 18KG 的数量为 1076
订单重量为 19KG 的数量为 928
订单重量为 20KG 的数量为 928
订单重量为 21KG 的数量为 902
订单重量为 22KG 的数量为 911
订单重量为 23KG 的数量为 818
订单重量为 24KG 的数量为 807
订单重量为 25KG 的数量为 784
订单重量为 26KG 的数量为 733
订单重量为 27KG 的数量为 688
订单重量为 28KG 的数量为 640
订单重量为 29KG 的数量为 642
订单重量为 30KG 的数量为 657
订单重量为 31KG 的数量为 607
订单重量为 32KG 的数量为 586
订单重量为 33KG 的数量为 607
订单重量为 34KG 的数量为 587
订单重量为 35KG 的数量为 558
订单重量为 36KG 的数量为 487
订单重量为 37KG 的数量为 552
订单重量为 38KG 的数量为 514
订单重量为 39KG 的数量为 492
订单重量为 40KG 的数量为 511
订单重量为 41KG 的数量为 484
订单重量为 42KG 的数量为 428
订单重量为 43KG 的数量为 426
订单重量为 44KG 的数量为 454
订单重量为 45KG 的数量为 408
订单重量为 46KG 的数量为 412
订单重量为 47KG 的数量为 407
订单重量为 48KG 的数量为 375
订单重量为 49KG 的数量为 379
订单重量为 50KG